# Connection to garmin connect

### Import

In [1]:
import datetime
from datetime import timezone
from datetime import date, timedelta
import json
import logging
import os
import sys
from getpass import getpass

import readchar
import requests
from garth.exc import GarthHTTPError

from garminconnect import (
    Garmin,
    GarminConnectAuthenticationError,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
)


### Connection

In [ ]:
# Configure debug logging
# logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
GARMIN_EMAIL = os.getenv("EMAIL")
GARMIN_PASSWORD = os.getenv("PASSWORD")

try:
    garmin = Garmin(GARMIN_EMAIL, GARMIN_PASSWORD)
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occurred during Garmin Connect Client init: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occurred during Garmin Connect Client init")
    quit()

In [4]:
print(GARMIN_EMAIL)

lucas.piquet@centrale.centralelille.fr


In [5]:
"""
Login to Garmin Connect portal
Only needed at start of your program
The library will try to relogin when session expires
"""
print("Login attempt: ...")
print("----------------------------------------------------------------------------------------")
try:
    garmin.login()
    print(f"Welcom to Garmin connect {garmin.get_full_name()}")
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occurred during Garmin Connect Client login: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occurred during Garmin Connect Client login")
    quit()

Login attempt: ...
----------------------------------------------------------------------------------------
Welcom to Garmin connect Piquet


### Fetch the first data

In [15]:
yesterday = date.today() - timedelta(days=1)
yesterday = yesterday.isoformat()
yesterday

'2025-03-14'

In [19]:
last_week = date.today() - timedelta(days=8)
last_week = last_week.isoformat()
last_week

'2025-03-07'

In [7]:
print(f"Data available : {garmin.get_stats(yesterday).keys()}")

Data available : dict_keys(['userProfileId', 'totalKilocalories', 'activeKilocalories', 'bmrKilocalories', 'wellnessKilocalories', 'burnedKilocalories', 'consumedKilocalories', 'remainingKilocalories', 'totalSteps', 'netCalorieGoal', 'totalDistanceMeters', 'wellnessDistanceMeters', 'wellnessActiveKilocalories', 'netRemainingKilocalories', 'userDailySummaryId', 'calendarDate', 'rule', 'uuid', 'dailyStepGoal', 'wellnessStartTimeGmt', 'wellnessStartTimeLocal', 'wellnessEndTimeGmt', 'wellnessEndTimeLocal', 'durationInMilliseconds', 'wellnessDescription', 'highlyActiveSeconds', 'activeSeconds', 'sedentarySeconds', 'sleepingSeconds', 'includesWellnessData', 'includesActivityData', 'includesCalorieConsumedData', 'privacyProtected', 'moderateIntensityMinutes', 'vigorousIntensityMinutes', 'floorsAscendedInMeters', 'floorsDescendedInMeters', 'floorsAscended', 'floorsDescended', 'intensityMinutesGoal', 'userFloorsAscendedGoal', 'minHeartRate', 'maxHeartRate', 'restingHeartRate', 'lastSevenDaysAvg

In [18]:
# garmin.get_steps_data(yesterday)
garmin.get_stats(yesterday)['totalKilocalories']


2712.0

In [11]:
garmin.get_user_summary(yesterday).keys()

dict_keys(['userProfileId', 'totalKilocalories', 'activeKilocalories', 'bmrKilocalories', 'wellnessKilocalories', 'burnedKilocalories', 'consumedKilocalories', 'remainingKilocalories', 'totalSteps', 'netCalorieGoal', 'totalDistanceMeters', 'wellnessDistanceMeters', 'wellnessActiveKilocalories', 'netRemainingKilocalories', 'userDailySummaryId', 'calendarDate', 'rule', 'uuid', 'dailyStepGoal', 'wellnessStartTimeGmt', 'wellnessStartTimeLocal', 'wellnessEndTimeGmt', 'wellnessEndTimeLocal', 'durationInMilliseconds', 'wellnessDescription', 'highlyActiveSeconds', 'activeSeconds', 'sedentarySeconds', 'sleepingSeconds', 'includesWellnessData', 'includesActivityData', 'includesCalorieConsumedData', 'privacyProtected', 'moderateIntensityMinutes', 'vigorousIntensityMinutes', 'floorsAscendedInMeters', 'floorsDescendedInMeters', 'floorsAscended', 'floorsDescended', 'intensityMinutesGoal', 'userFloorsAscendedGoal', 'minHeartRate', 'maxHeartRate', 'restingHeartRate', 'lastSevenDaysAvgRestingHeartRate'

In [17]:
garmin.get_floors(yesterday)

{'startTimestampGMT': '2025-03-13T23:00:00.0',
 'endTimestampGMT': '2025-03-14T23:00:00.0',
 'startTimestampLocal': '2025-03-14T00:00:00.0',
 'endTimestampLocal': '2025-03-15T00:00:00.0',
 'floorsValueDescriptorDTOList': [],
 'floorValuesArray': []}

In [20]:
garmin.get_daily_steps(last_week, yesterday)

[{'calendarDate': '2025-03-07',
  'totalSteps': None,
  'totalDistance': None,
  'stepGoal': None},
 {'calendarDate': '2025-03-08',
  'totalSteps': None,
  'totalDistance': None,
  'stepGoal': None},
 {'calendarDate': '2025-03-09',
  'totalSteps': None,
  'totalDistance': None,
  'stepGoal': None},
 {'calendarDate': '2025-03-10',
  'totalSteps': 11997,
  'totalDistance': 11996,
  'stepGoal': 14660},
 {'calendarDate': '2025-03-11',
  'totalSteps': 13618,
  'totalDistance': 14350,
  'stepGoal': 14660},
 {'calendarDate': '2025-03-12',
  'totalSteps': 2200,
  'totalDistance': 1730,
  'stepGoal': 14660},
 {'calendarDate': '2025-03-13',
  'totalSteps': 12691,
  'totalDistance': 14000,
  'stepGoal': 14660},
 {'calendarDate': '2025-03-14',
  'totalSteps': 11577,
  'totalDistance': 12077,
  'stepGoal': 14660}]

In [21]:
garmin.get_heart_rates(yesterday).keys()

dict_keys(['userProfilePK', 'calendarDate', 'startTimestampGMT', 'endTimestampGMT', 'startTimestampLocal', 'endTimestampLocal', 'maxHeartRate', 'minHeartRate', 'restingHeartRate', 'lastSevenDaysAvgRestingHeartRate', 'heartRateValueDescriptors', 'heartRateValues'])

In [22]:
garmin.get_all_day_stress(yesterday)['bodyBatteryValuesArray'][:10]

[[1741906800000, 'MEASURED', 29, 2.0],
 [1741906980000, 'MEASURED', 30, 2.0],
 [1741907160000, 'MEASURED', 30, 2.0],
 [1741907340000, 'MEASURED', 31, 2.0],
 [1741907520000, 'MEASURED', 31, 2.0],
 [1741907700000, 'MEASURED', 32, 2.0],
 [1741907880000, 'MEASURED', 33, 2.0],
 [1741908060000, 'MEASURED', 33, 2.0],
 [1741908240000, 'MEASURED', 34, 2.0],
 [1741908420000, 'MEASURED', 34, 2.0]]

In [23]:
garmin.get_personal_record()[2]

{'id': 2444970671,
 'typeId': 4,
 'status': 'ACCEPTED',
 'activityId': 15585601288,
 'activityName': 'Paris Course à pied',
 'activityType': 'running',
 'activityStartDateTimeInGMT': 1716708641000,
 'actStartDateTimeInGMTFormatted': '2024-05-26T07:30:41.0',
 'activityStartDateTimeLocal': 1716715841000,
 'activityStartDateTimeLocalFormatted': '2024-05-26T09:30:41.0',
 'value': 2320.009033203125,
 'prStartTimeGmt': 1716708641000,
 'prStartTimeGmtFormatted': '2024-05-26T07:30:41.0',
 'prStartTimeLocal': None,
 'prStartTimeLocalFormatted': None,
 'prTypeLabelKey': None,
 'poolLengthUnit': None}

# Example

### From medium

https://medium.com/@datascienceandvisualisation/a-walkthrough-of-downloading-and-analysing-garmin-connect-data-using-python-b38c51f2fd85

## From garmin connect

https://pypi.org/project/garminconnect/0.1.21/#description